# Description

**TODO**

# Modules

In [1]:
library(reticulate)
pd <- import("pandas")

# Settings

In [2]:
# reference panel
REFERENCE_PANEL <- "GTEX_V8"
# REFERENCE_PANEL = "1000G"

# prediction models
## mashr
EQTL_MODEL <- "MASHR"

# ## elastic net
# EQTL_MODEL = "ELASTIC_NET"

chromosome <- NULL

In [3]:
# Parameters
chromosome = 1
REFERENCE_PANEL = "GTEX_V8"
EQTL_MODEL = "MASHR"


In [4]:
paste0("Using reference panel: ", REFERENCE_PANEL)

[1] "Using reference panel: GTEX_V8"

In [5]:
paste0("Using eQTL model: ", EQTL_MODEL)

[1] "Using eQTL model: MASHR"

In [6]:
# chromosome must be provided as parameter
stopifnot(!is.null(chromosome))

# Paths

In [7]:
GENE_CORRS_DIR <- Sys.getenv("PHENOPLIER_PHENOMEXCAN_LD_BLOCKS_GENE_CORRS_DIR")
IRdisplay::display(GENE_CORRS_DIR)

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs"

In [8]:
INPUT_DIR <- file.path(GENE_CORRS_DIR, tolower(REFERENCE_PANEL), tolower(EQTL_MODEL), "by_chr")
IRdisplay::display(INPUT_DIR)

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs/gtex_v8/mashr/by_chr"

In [9]:
INPUT_FILE <- file.path(INPUT_DIR, paste0("gene_corrs-chr", chromosome, ".pkl"))
IRdisplay::display(INPUT_FILE)
stopifnot(file.exists(INPUT_FILE))

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs/gtex_v8/mashr/by_chr/gene_corrs-chr1.pkl"

In [10]:
OUTPUT_DIR <- file.path(INPUT_DIR, "corrected_positive_definite")
IRdisplay::display(OUTPUT_DIR)
dir.create(OUTPUT_DIR, recursive = TRUE, showWarnings = FALSE)

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs/gtex_v8/mashr/by_chr/corrected_positive_definite"

In [11]:
OUTPUT_FILE <- file.path(OUTPUT_DIR, paste0("gene_corrs-chr", chromosome, ".pkl"))
IRdisplay::display(OUTPUT_FILE)
if (file.exists(OUTPUT_FILE)) {
  IRdisplay::display("Output file exists, it will be overwritten")
}

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs/gtex_v8/mashr/by_chr/corrected_positive_definite/gene_corrs-chr1.pkl"

[1] "Output file exists, it will be overwritten"

# Functions

In [12]:
# taken from https://www.r-bloggers.com/2013/08/correcting-a-pseudo-correlation-matrix-to-be-positive-semidefinite/
# TODO: add documentation
CorrectCM <- function(CM, p = 0) {
  n <- dim(var(CM))[1L]
  E <- eigen(CM)
  CM1 <- E$vectors %*% tcrossprod(diag(pmax(E$values, p), n), E$vectors)
  Balance <- diag(1 / sqrt(diag(CM1)))
  CM2 <- Balance %*% CM1 %*% Balance
  return(CM2)
}

In [13]:
is_positive_definite <- function(data) {
  eigenvalues <- eigen(data)$values
  nonpositive_eigenvalues <- eigenvalues[eigenvalues <= 0]

  if (length(nonpositive_eigenvalues) > 0) {
    IRdisplay::display("We need to correct the data and make the matrix positive definite")
    return(FALSE)
  } else {
    IRdisplay::display("Matrix is already positive definite!")
    return(TRUE)
  }
}

# Load data

In [14]:
gene_corrs <- pd$read_pickle(INPUT_FILE)

In [15]:
dim(gene_corrs)

[1] 700 700

In [16]:
head(gene_corrs[1:10, 1:10])

,ENSG00000188976,ENSG00000188290,ENSG00000187608,ENSG00000188157,ENSG00000186891,ENSG00000186827,ENSG00000176022,ENSG00000160087,ENSG00000131584,ENSG00000169962
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000188976,1.00000000,0.02361128,0.051490837,-0.34217863,0.023875079,0.05151428,-0.11406162,-0.04373261,0.017677782,0.068202071
ENSG00000188290,0.02361128,1.00000000,0.136524599,-0.07697149,0.092037617,-0.05575377,-0.06365866,-0.06995675,0.103245595,-0.060622698
ENSG00000187608,0.05149084,0.13652460,1.000000000,0.01305981,-0.009458447,-0.07905110,-0.24629385,-0.06376631,0.016577053,-0.097012421
ENSG00000188157,-0.34217863,-0.07697149,0.013059813,1.00000000,-0.058856618,-0.14024394,-0.25159589,-0.04346579,-0.005467761,-0.034212459
ENSG00000186891,0.02387508,0.09203762,-0.009458447,-0.05885662,1.000000000,0.09812693,0.27406428,-0.19015347,0.277689351,-0.034433500
ENSG00000186827,0.05151428,-0.05575377,-0.079051100,-0.14024394,0.098126927,1.00000000,0.20819153,0.16215540,0.041733146,0.001576558


# Check positive definiteness

In [17]:
is_positive_definite(gene_corrs)

[1] "We need to correct the data and make the matrix positive definite"

[1] FALSE

In [18]:
# see eigenvalues
eigenvalues <- eigen(gene_corrs)$values

In [19]:
nonpositive_eigenvalues <- eigenvalues[eigenvalues <= 0]
IRdisplay::display(length(nonpositive_eigenvalues))
IRdisplay::display(nonpositive_eigenvalues)

[1] 81

[1] -0.01569553 -0.02083276 -0.03343062 -0.04161605 -0.04672675 -0.04879697
 [7] -0.04938975 -0.07312427 -0.08087685 -0.08930701 -0.08981065 -0.09617212
[13] -0.10233187 -0.10242506 -0.10707568 -0.11134262 -0.12752268 -0.13316349
[19] -0.13581378 -0.15603728 -0.16973342 -0.17523257 -0.17656838 -0.18449245
[25] -0.19430074 -0.19630455 -0.21505317 -0.22378339 -0.24611695 -0.26885035
[31] -0.28812097 -0.30011771 -0.30640459 -0.31058875 -0.31735291 -0.31844577
[37] -0.32539933 -0.37350997 -0.38998489 -0.39767154 -0.42336726 -0.44710853
[43] -0.46410515 -0.47638984 -0.53545631 -0.53821114 -0.55739620 -0.57024090
[49] -0.58861318 -0.60592451 -0.61306032 -0.61597469 -0.63957565 -0.67626856
[55] -0.68535649 -0.77661660 -0.79695269 -0.82713322 -0.85163609 -0.85608648
[61] -0.96391462 -0.99784893 -1.02686898 -1.02868449 -1.05120024 -1.17731129
[67] -1.24559734 -1.27396310 -1.33315920 -1.42582603 -1.49779289 -1.63034189
[73] -1.66344663 -1.93073880 -2.10599786 -2.35925101 -2.78943258 -3.05668598
[79] -3.31178549 -3.68830248 -4.79597807

In [20]:
if (length(eigenvalues[eigenvalues <= 0]) == 0) { quit() }

# Make matrix positive definite if needed

In [21]:
gene_corrs_corrected <- CorrectCM(gene_corrs, 1)

In [22]:
dimnames(gene_corrs_corrected)[[1]] <- rownames(gene_corrs)

In [23]:
dimnames(gene_corrs_corrected)[[2]] <- colnames(gene_corrs)

In [24]:
gene_corrs_corrected <- as.data.frame(gene_corrs_corrected)

In [25]:
dim(gene_corrs_corrected)

[1] 700 700

# Check positive definiteness of corrected matrix

In [26]:
is_positive_definite(gene_corrs_corrected)

[1] "Matrix is already positive definite!"

[1] TRUE

In [27]:
# see eigenvalues
eigenvalues <- eigen(gene_corrs_corrected)$values

In [28]:
nonpositive_eigenvalues <- eigenvalues[eigenvalues <= 0]
IRdisplay::display(length(nonpositive_eigenvalues))
IRdisplay::display(nonpositive_eigenvalues)

[1] 0

numeric(0)

In [29]:
stopifnot(length(eigenvalues[eigenvalues <= 0]) == 0)

In [30]:
# quick and visual comparison of the two matrices
IRdisplay::display(head(gene_corrs[1:10, 1:10]))
IRdisplay::display(head(gene_corrs_corrected[1:10, 1:10]))

,ENSG00000188976,ENSG00000188290,ENSG00000187608,ENSG00000188157,ENSG00000186891,ENSG00000186827,ENSG00000176022,ENSG00000160087,ENSG00000131584,ENSG00000169962
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000188976,1.00000000,0.02361128,0.051490837,-0.34217863,0.023875079,0.05151428,-0.11406162,-0.04373261,0.017677782,0.068202071
ENSG00000188290,0.02361128,1.00000000,0.136524599,-0.07697149,0.092037617,-0.05575377,-0.06365866,-0.06995675,0.103245595,-0.060622698
ENSG00000187608,0.05149084,0.13652460,1.000000000,0.01305981,-0.009458447,-0.07905110,-0.24629385,-0.06376631,0.016577053,-0.097012421
ENSG00000188157,-0.34217863,-0.07697149,0.013059813,1.00000000,-0.058856618,-0.14024394,-0.25159589,-0.04346579,-0.005467761,-0.034212459
ENSG00000186891,0.02387508,0.09203762,-0.009458447,-0.05885662,1.000000000,0.09812693,0.27406428,-0.19015347,0.277689351,-0.034433500
ENSG00000186827,0.05151428,-0.05575377,-0.079051100,-0.14024394,0.098126927,1.00000000,0.20819153,0.16215540,0.041733146,0.001576558


,ENSG00000188976,ENSG00000188290,ENSG00000187608,ENSG00000188157,ENSG00000186891,ENSG00000186827,ENSG00000176022,ENSG00000160087,ENSG00000131584,ENSG00000169962
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000188976,1.00000000,0.062865932,-0.01084716,-0.13166975,0.03709920,0.019131155,-0.001807355,-0.05621752,0.026165101,-0.0003533378
ENSG00000188290,0.06286593,1.000000000,0.08933150,-0.03363005,0.03708635,0.006598331,-0.003540029,-0.02882759,0.057050439,-0.0354044988
ENSG00000187608,-0.01084716,0.089331499,1.00000000,0.00100939,-0.03557471,-0.022576077,-0.084672840,-0.04583607,-0.022564390,-0.0258651220
ENSG00000188157,-0.13166975,-0.033630047,0.00100939,1.00000000,-0.08416525,-0.081193321,-0.123104554,-0.01348113,-0.004321667,0.0098954284
ENSG00000186891,0.03709920,0.037086349,-0.03557471,-0.08416525,1.00000000,0.061211912,0.175984655,-0.06478961,0.155956436,-0.0311701177
ENSG00000186827,0.01913116,0.006598331,-0.02257608,-0.08119332,0.06121191,1.000000000,0.207030575,0.07861739,0.048532776,-0.0083645054


Both matrices should "look" similar. We are not interested in perfectly accurate correlation values (they are already inaccurate).

# Save

In [31]:
py_save_object(gene_corrs_corrected, OUTPUT_FILE)